In [2]:
import mlflow
from mlflow.models import ModelSignature
from mlflow.types.schema import Schema, ColSpec, ParamSchema, ParamSpec
import requests
import numpy as np

class MathModel(mlflow.pyfunc.PythonModel):
    def predict(self, ctx, model_input, params):
        n1 = model_input['number_1'][0]
        n2 = model_input['number_2'][0]
        operation = params["operation"]

        return eval("{0} {2} {1}".format(n1, n2, operation))

In [3]:
params_schema = ParamSchema(
    [
        ParamSpec("operation", "string", " + ")
    ]
)

input_schema = Schema(
    [
        ColSpec("integer", "number_1"),
        ColSpec("integer", "number_2")
    ]
)

output_schema = Schema(
    [
        ColSpec("string","result")
    ]
)

signature = ModelSignature(inputs=input_schema, outputs=output_schema, params = params_schema)

mlflow.set_experiment("math_model")
with mlflow.start_run() as run:
    model_info = mlflow.pyfunc.log_model(
        python_model=MathModel(), 
        artifact_path="math_model",
        signature=signature,
        registered_model_name="math_model" # Register the model
    )

client = mlflow.MlflowClient()
model_metadata = client.get_latest_versions("math_model", stages=["None"])
latest_model_version = model_metadata[0].version

Registered model 'math_model' already exists. Creating a new version of this model...
2024/01/26 16:30:02 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: math_model, version 5
Created version '5' of model 'math_model'.


In [9]:
mlflow.models.get_model_info(f"models:/math_model/{latest_model_version}").signature

inputs: 
  ['number_1': integer, 'number_2': integer]
outputs: 
  ['result': string]
params: 
  ['operation': string (default:  + )]

# Usage

In the following are two ways to use the registered models. 

## Loading the model

Using pyfunc.load_model you can test if your model is working well, but it doesn`t involves nothing of the model services feature.

In order to use it, change the version of model_uri to the latests version you deployed

In [8]:
model = mlflow.pyfunc.load_model(model_uri=f"models:/math_model/{latest_model_version}")
n1 = 2
n2 = 10
op = "*"

model.predict({"number_1": np.array([n1], dtype=np.int32), "number_2":np.array([n2], dtype=np.int32)}, params={"operation":op})

20

## Model services

Go to the published services page and create a model, run it and wait until the running state. Change the URL field below to the url of the published services.

In [5]:
# Model service 🤘

url = "https://10.137.137.5:5000"+"/invocations"
data = {
    "inputs":{
        "number_1":[2],
        "number_2":[10]
    },
    "params":{
        "operation": "*"
    }
}

headers = {'Content-type': 'application/json'}
x = requests.post(url, json = data, verify=False, headers=headers)

print(x.text)

{"predictions": {"flowerName": "sunflower", "do_work": false}}


/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.137.137.5'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
